<a href="https://colab.research.google.com/github/Mageed-Ghaleb/OptimizationSystems-Course/blob/main/IP_and_MIP_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 4 Companion Notebook (Optional)
**Integer & Mixed-Integer Programming — Modeling Patterns + Solvers**

This notebook is an optional companion to **Lecture 4 (IP/MIP + combinatorial intro)**.
It demonstrates common formulation patterns and how to solve them using:

- **Pyomo + HiGHS** (MILP/MIP)
- **OR-Tools CP-SAT** (logic-heavy 0–1 models and combinatorial structure)

You can reuse these templates for Winston Chapter 9 homework problems.

In [1]:
# Colab setup (run once)
!pip -q install pyomo highspy ortools pandas numpy

import math
import pandas as pd
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.


## 1) Helper: Solve Pyomo MILPs with HiGHS
We use HiGHS (fast open-source MILP solver) through Pyomo.
If HiGHS is not detected, re-run the install cell above.

In [2]:
from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Binary,
    Set, minimize, maximize, value
)
from pyomo.opt import SolverFactory

def solve_with_highs(model, tee=False):
    for name in ["appsi_highs", "highs"]:
        try:
            solver = SolverFactory(name)
            if solver is not None and solver.available():
                res = solver.solve(model, tee=tee)
                return name, res
        except Exception:
            pass
    raise RuntimeError("No HiGHS solver found. Try: pip install highspy pyomo")

## 2) Pattern A: Fixed-charge / Setup cost (binary + Big-M)

**When used:** If an activity costs a fixed amount if it is used at all.

**Template**
- activity level: `x >= 0`
- activation: `y in {0,1}`
- linking constraint: `x <= M*y`
- objective includes `fixed*y + variable*x`

### Mini case
A firm can run production on up to 2 lines. Each line has:
- a fixed startup cost
- unit cost
- capacity
Meet demand at minimum cost.

In [3]:
### Code (Fixed-charge in Pyomo)
# Data (edit freely)
demand = 120
lines = ["Line1", "Line2"]

fixed = {"Line1": 800, "Line2": 500}
unit  = {"Line1": 9,   "Line2": 11}
cap   = {"Line1": 100, "Line2": 80}

# Tight Big-M choice: use capacity per line
M = cap

m = ConcreteModel()
m.L = Set(initialize=lines)

m.x = Var(m.L, domain=NonNegativeReals)  # production
m.y = Var(m.L, domain=Binary)            # 1 if line is used

m.obj = Objective(
    expr=sum(fixed[l]*m.y[l] + unit[l]*m.x[l] for l in m.L),
    sense=minimize
)

m.demand = Constraint(expr=sum(m.x[l] for l in m.L) >= demand)
m.cap = Constraint(m.L, rule=lambda mm, l: mm.x[l] <= M[l]*mm.y[l])

solver_name, res = solve_with_highs(m)
print("Solved with:", solver_name)
print("Objective value =", value(m.obj))
for l in lines:
    print(l, "x =", value(m.x[l]), "y =", value(m.y[l]))

Solved with: appsi_highs
Objective value = 2420.0
Line1 x = 100.0 y = 1.0
Line2 x = 20.0 y = 1.0


### Interpretation
- If `y[line]=0`, then `x[line]=0` (line not used).
- If `y[line]=1`, the line can produce up to its capacity.
- Big-M should be as tight as possible (capacity is a standard tight choice here).

## 3) Same fixed-charge model in OR-Tools CP-SAT

CP-SAT uses integer variables. Here production is in integer units.

In [4]:
### Code (Fixed-charge in CP-SAT)
from ortools.sat.python import cp_model

model = cp_model.CpModel()

x = {l: model.NewIntVar(0, cap[l], f"x[{l}]") for l in lines}
y = {l: model.NewBoolVar(f"y[{l}]") for l in lines}

model.Add(sum(x[l] for l in lines) >= demand)

for l in lines:
    model.Add(x[l] <= cap[l]*y[l])

model.Minimize(sum(fixed[l]*y[l] + unit[l]*x[l] for l in lines))

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
status = solver.Solve(model)

print("Status:", solver.StatusName(status))
print("Objective =", solver.ObjectiveValue())
for l in lines:
    print(l, "x =", solver.Value(x[l]), "y =", solver.Value(y[l]))

Status: OPTIMAL
Objective = 2420.0
Line1 x = 100 y = 1
Line2 x = 20 y = 1


## 4) Pattern B: Logical constraints in 0–1 IP

Common patterns:
- At most K chosen: `sum(y_i) <= K`
- Mutual exclusion: `y_a + y_b <= 1`
- If A then B: `y_a <= y_b`

### Mini case: choose projects
Maximize benefit subject to budget + logical rules.

In [5]:
### Code (Logical 0–1 selection with CP-SAT)
projects = ["A","B","C","D","E"]
benefit = {"A":12, "B":10, "C":9, "D":7, "E":6}
cost    = {"A":8,  "B":6,  "C":5, "D":4, "E":3}
budget = 15

model = cp_model.CpModel()
y = {p: model.NewBoolVar(f"y[{p}]") for p in projects}

model.Add(sum(cost[p]*y[p] for p in projects) <= budget)

# Logical rules (edit freely)
model.Add(sum(y[p] for p in projects) <= 3)     # at most 3 projects
model.Add(y["A"] <= y["D"])                     # if A then D
model.Add(y["B"] + y["C"] <= 1)                 # not both B and C
model.Add(y["E"] + y["A"] <= 1)                 # E excludes A

model.Maximize(sum(benefit[p]*y[p] for p in projects))

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
status = solver.Solve(model)

chosen = [p for p in projects if solver.Value(y[p]) == 1]
print("Status:", solver.StatusName(status))
print("Chosen:", chosen)
print("Total benefit:", solver.ObjectiveValue())
print("Total cost:", sum(cost[p] for p in chosen))

Status: OPTIMAL
Chosen: ['B', 'D', 'E']
Total benefit: 23.0
Total cost: 13


## 5) Pattern C: 0–1 Knapsack (classic combinatorial model)

Choose items to maximize value subject to a capacity constraint.
This is a canonical example that connects directly to branch-and-bound intuition.

In [6]:
### Code (Knapsack with CP-SAT)
items = ["i1","i2","i3","i4","i5","i6"]
value_w = {"i1":12,"i2":10,"i3":8,"i4":11,"i5":7,"i6":6}
weight  = {"i1":7, "i2":6, "i3":5,"i4":9, "i5":4, "i6":3}
capacity = 18

model = cp_model.CpModel()
x = {i: model.NewBoolVar(f"x[{i}]") for i in items}

model.Add(sum(weight[i]*x[i] for i in items) <= capacity)
model.Maximize(sum(value_w[i]*x[i] for i in items))

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
status = solver.Solve(model)

chosen = [i for i in items if solver.Value(x[i]) == 1]
print("Status:", solver.StatusName(status))
print("Chosen:", chosen)
print("Total value:", solver.ObjectiveValue())
print("Total weight:", sum(weight[i] for i in chosen))

Status: OPTIMAL
Chosen: ['i2', 'i3', 'i5', 'i6']
Total value: 31.0
Total weight: 18


## 6) Mini TSP (MILP) using MTZ subtour elimination (Pyomo)

This demonstrates how a routing problem becomes a MILP:
- binary arc variables x[i,j]
- degree constraints (one in, one out)
- MTZ constraints to prevent subtours

This is intended for small n (6–10). For larger routing, CP-SAT is often preferable.

In [7]:
import random

random.seed(7)
n = 6
cities = list(range(n))

# Symmetric random distances (replace with real data if desired)
dist = {(i,j): 0 for i in cities for j in cities}
for i in cities:
    for j in cities:
        if i == j:
            dist[(i,j)] = 0
        elif j < i:
            dist[(i,j)] = dist[(j,i)]
        else:
            dist[(i,j)] = random.randint(10, 80)

m = ConcreteModel()
m.N = Set(initialize=cities)

m.x = Var(m.N, m.N, domain=Binary)
m.u = Var(m.N, domain=NonNegativeReals, bounds=(0, n-1))

m.obj = Objective(
    expr=sum(dist[(i,j)]*m.x[i,j] for i in cities for j in cities if i != j),
    sense=minimize
)

m.out = Constraint(m.N, rule=lambda mm, i: sum(mm.x[i,j] for j in cities if j != i) == 1)
m.inn = Constraint(m.N, rule=lambda mm, j: sum(mm.x[i,j] for i in cities if i != j) == 1)
m.noself = Constraint(m.N, rule=lambda mm, i: mm.x[i,i] == 0)

m.u_fix = Constraint(expr=m.u[0] == 0)

def mtz_rule(mm, i, j):
    if i == j or i == 0 or j == 0:
        return Constraint.Skip
    return mm.u[i] - mm.u[j] + (n-1)*mm.x[i,j] <= n-2

m.mtz = Constraint(m.N, m.N, rule=mtz_rule)

solver_name, res = solve_with_highs(m)
print("Solved with:", solver_name)
print("Objective value =", value(m.obj))

# Extract tour successor mapping
succ = {}
for i in cities:
    for j in cities:
        if i != j and value(m.x[i,j]) > 0.5:
            succ[i] = j

tour = [0]
while True:
    nxt = succ[tour[-1]]
    if nxt == 0:
        tour.append(0)
        break
    tour.append(nxt)

print("Tour:", tour)

Solved with: appsi_highs
Objective value = 118.99999999999997
Tour: [0, 4, 3, 1, 5, 2, 0]


## 7) Optional practice tasks
1) Add a **minimum-run constraint** to the fixed-charge model: if a line is started, it must produce at least L units.
2) Add an **either-or** rule to the project selection (choose A or B, but not both).
3) Increase TSP to 8 cities and compare solve time (when does MILP get hard?).
4) Rebuild one Winston Chapter 9 homework problem by replacing the toy data with the homework data.